<a href="https://colab.research.google.com/github/Hayeonggg/CAU-IIPL-2024-Internship/blob/main/week7_Translate_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 데이터 처리하기

In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 47.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.8 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [ ]:
import torch
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast
from datasets import load_dataset
from torch.utils.data import Dataset, DataLoader

#토크나이저
model_name = "facebook/mbart-large-50-many-to-many-mmt"
tokenizer = MBart50TokenizerFast.from_pretrained(model_name)
model = MBartForConditionalGeneration.from_pretrained(model_name)

tokenizer.src_lang = 'en_XX'
tokenizer.tgt_lang='de_DE'
#데이터셋 로드
ds = load_dataset("bentrevett/multi30k")

# 데이터셋 클래스 정의
class TranslationDataset(Dataset):
    def __init__(self, dataset, tokenizer, max_len=128):
        self.dataset = dataset
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        src_text = self.dataset[idx]['en']
        tgt_text = self.dataset[idx]['de']
        src_tokens = self.tokenizer(src_text, padding='max_length', truncation=True, max_length=self.max_len, return_tensors="pt", add_special_tokens=True)
        tgt_tokens = self.tokenizer(tgt_text, padding='max_length', truncation=True, max_length=self.max_len, return_tensors="pt", add_special_tokens=True)

        return {
            'src_input_ids': src_tokens['input_ids'].squeeze(),
            'tgt_input_ids': tgt_tokens['input_ids'].squeeze(),
            'src_attention_mask': src_tokens['attention_mask'].squeeze(),
            'tgt_attention_mask': tgt_tokens['attention_mask'].squeeze()
        }


#데이터셋-> 전처리된 데이터셋으로 변환
train_dataset = TranslationDataset(ds['train'],tokenizer)
val_dataset = TranslationDataset(ds['validation'],tokenizer)
test_dataset = TranslationDataset(ds['test'],tokenizer)

train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=8, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=8, shuffle=False)


#실제 토큰화된 결과 확인
for batch in train_dataset:
  print(batch)
  break
#<BOS> 250004 / <EOS> 2 / <PAD> 1

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/529 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/649 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

...

test.jsonl:   0%|          | 0.00/156k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/29000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1014 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1000 [00:00<?, ? examples/s]

{'src_input_ids': tensor([250004,  32964,  27150,      4,  22392,  11280,      7,    621,  50782,
         43573,   5941,    373, 114942,      5,      2,      1,      1,      1,
             1,      1,      1,      1,      1,      1,      1,      1,      1,
             1,      1,      1,      1,      1,      1,      1,      1,      1,
             1,      1,      1,      1,      1,      1,      1,      1,      1,
             1,      1,      1,      1,      1,      1,      1,      1,      1,
             1,      1,      1,      1,      1,      1,      1,      1,      1,
             1,      1,      1,      1,      1,      1,      1,      1,      1,
             1,      1,      1,      1,      1,      1,      1,      1,      1,
             1,      1,      1,      1,      1,      1,      1,      1,      1,
             1,      1,      1,      1,      1,      1,      1,      1,      1,
             1,      1,      1,      1,      1,      1,      1,      1,      1,
             1,      1

In [ ]:
for batch in train_dataloader:
  print(batch)
  break

{'src_input_ids': tensor([[250004,     62,    332,  ...,      1,      1,      1],
        [250004,     62,    332,  ...,      1,      1,      1],
        [250004,     62,  30906,  ...,      1,      1,      1],
        ...,
        [250004,     62,      6,  ...,      1,      1,      1],
        [250004,  74057,      7,  ...,      1,      1,      1],
        [250004,     62,  11280,  ...,      1,      1,      1]]), 'tgt_input_ids': tensor([[250004,   2991,  14912,  ...,      1,      1,      1],
        [250004,   2991,  14912,  ...,      1,      1,      1],
        [250004,   2991,  30906,  ...,      1,      1,      1],
        ...,
        [250004,  12080,  28345,  ...,      1,      1,      1],
        [250004,  55065,   2235,  ...,      1,      1,      1],
        [250004,   2991,  14912,  ...,      1,      1,      1]]), 'src_attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 

# Transformer

In [ ]:
from torch import Tensor
import torch
import torch.nn as nn
from torch.nn import Transformer
import math
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 단어 순서 개념(notion)을 토큰 임베딩에 도입하기 위한 위치 인코딩(positional encoding)을 위한 헬퍼 모듈(Module)
class PositionalEncoding(nn.Module):
    def __init__(self,
                 emb_size: int,
                 dropout: float,
                 maxlen: int = 5000):
        super(PositionalEncoding, self).__init__()
        den = torch.exp(- torch.arange(0, emb_size, 2)* math.log(10000) / emb_size)
        pos = torch.arange(0, maxlen).reshape(maxlen, 1)
        pos_embedding = torch.zeros((maxlen, emb_size))
        pos_embedding[:, 0::2] = torch.sin(pos * den)
        pos_embedding[:, 1::2] = torch.cos(pos * den)
        pos_embedding = pos_embedding.unsqueeze(-2)

        self.dropout = nn.Dropout(dropout)
        self.register_buffer('pos_embedding', pos_embedding)

    def forward(self, token_embedding: Tensor):
        return self.dropout(token_embedding + self.pos_embedding[:token_embedding.size(0), :])

# 입력 인덱스의 텐서를 해당하는 토큰 임베딩의 텐서로 변환하기 위한 헬퍼 모듈(Module)
class TokenEmbedding(nn.Module):
    def __init__(self, vocab_size: int, emb_size):
        super(TokenEmbedding, self).__init__()
        self.embedding = nn.Embedding(vocab_size, emb_size)
        self.emb_size = emb_size

    def forward(self, tokens: Tensor):
        return self.embedding(tokens.long()) * math.sqrt(self.emb_size)


class Seq2SeqTransformer(nn.Module):
    def __init__(self,
                 num_encoder_layers: int,
                 num_decoder_layers: int,
                 emb_size: int,
                 nhead: int,
                 src_vocab_size: int,
                 tgt_vocab_size: int,
                 dim_feedforward: int = 512,
                 dropout: float = 0.1):
        super(Seq2SeqTransformer, self).__init__()
        self.transformer = Transformer(d_model=emb_size,
                                       nhead=nhead,
                                       num_encoder_layers=num_encoder_layers,
                                       num_decoder_layers=num_decoder_layers,
                                       dim_feedforward=dim_feedforward,
                                       dropout=dropout)
        self.generator = nn.Linear(emb_size, tgt_vocab_size)#출력 생성기
        self.src_tok_emb = TokenEmbedding(src_vocab_size, emb_size)#source 토큰 임베딩
        self.tgt_tok_emb = TokenEmbedding(tgt_vocab_size, emb_size)#target 토큰 임베딩
        self.positional_encoding = PositionalEncoding(#위치 인코딩
            emb_size, dropout=dropout)

    #인코더-디코더 각 과정/훈련데이터 처리
    def forward(self,
                src: Tensor,
                trg: Tensor,
                src_mask: Tensor,
                tgt_mask: Tensor,
                src_padding_mask: Tensor,
                tgt_padding_mask: Tensor,
                memory_key_padding_mask: Tensor):
        src_emb = self.positional_encoding(self.src_tok_emb(src))
        tgt_emb = self.positional_encoding(self.tgt_tok_emb(trg))
        #source 문장을 처리한 후, target문장을 디코더로 처리?
        outs = self.transformer(src_emb, tgt_emb, src_mask, tgt_mask, None,
                                src_padding_mask, tgt_padding_mask, memory_key_padding_mask)#mask 적용
        return self.generator(outs)#최종적으로 각 타겟 단어에 대한 확률분포 반환/임베딩 차원->선형변환 적용->확률분포

    #추론(예측) 과정
    #인코딩
    def encode(self, src: Tensor, src_mask: Tensor):#source 문장 인코딩
        return self.transformer.encoder(self.positional_encoding(
                            self.src_tok_emb(src)), src_mask)
    #디코딩
    def decode(self, tgt: Tensor, memory: Tensor, tgt_mask: Tensor):#target 문장 디코딩
        return self.transformer.decoder(self.positional_encoding(
                          self.tgt_tok_emb(tgt)), memory,
                          tgt_mask)

In [ ]:
#마스킹 생성
def generate_square_subsequent_mask(sz):
    mask = (torch.triu(torch.ones((sz, sz), device=DEVICE)) == 1).transpose(0, 1)#상삼각행렬 생성(순차적인 마스킹 적용)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))#-무한대로 마스킹 적용
    return mask

#마스크 설정
def create_mask(src, tgt):
   #[0]: 배치크기, [1]:시퀀스 길이
    tgt_seq_len = tgt.shape[0]-1#128-1


    #어텐션 masking(디코더)
    tgt_mask = generate_square_subsequent_mask(tgt_seq_len).type(torch.float32).to(DEVICE)

    #패딩 토큰에 대한 masking(인코더/디코더)
    src_padding_mask = (src == tokenizer.pad_token_id).to(torch.bool).transpose(0,1)#[src_seq_len, batch_size]
    tgt_padding_mask = (tgt[:-1,:] == tokenizer.pad_token_id).to(torch.bool).transpose(0,1)

    #차원 확인용
    #print('tgt_seq_len',tgt_seq_len, '  tgt_mask',tgt_mask.shape  )
    #print('\n',src_padding_mask.shape, '\n',tgt_padding_mask.shape)

    return None, tgt_mask, src_padding_mask, tgt_padding_mask #src_mask는 반환하지 않음

In [ ]:
torch.manual_seed(0)

SRC_VOCAB_SIZE = tokenizer.vocab_size
TGT_VOCAB_SIZE = tokenizer.vocab_size
EMB_SIZE = 512#512
NHEAD = 4#8
FFN_HID_DIM = 512#2048
BATCH_SIZE = 8
NUM_ENCODER_LAYERS = 2 #6
NUM_DECODER_LAYERS = 2 #6

transformer = Seq2SeqTransformer(NUM_ENCODER_LAYERS, NUM_DECODER_LAYERS, EMB_SIZE,
                                 NHEAD, SRC_VOCAB_SIZE, TGT_VOCAB_SIZE, FFN_HID_DIM)

model = transformer.to(DEVICE)

"""for p in transformer.parameters():
    if p.dim() > 1:
        nn.init.xavier_uniform_(p)

transformer = transformer.to(DEVICE)

BOS_IDX = 250004
EOS_IDX=2
PAD_IDX= 1"""

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:307: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


'for p in transformer.parameters():\n    if p.dim() > 1:\n        nn.init.xavier_uniform_(p)\n\ntransformer = transformer.to(DEVICE)\n\nBOS_IDX = 250004\nEOS_IDX=2\nPAD_IDX= 1'

# 확인하기

In [ ]:
#1. 패딩 마스크 및 데이터 전처리 확인:
def check_padding_and_eos(src, tgt, tokenizer):
    # 패딩 토큰 및 eos 토큰의 인덱스
    pad_token_id = tokenizer.pad_token_id
    eos_token_id = tokenizer.eos_token_id

    # 패딩 마스크 확인
    print("패딩 마스크 확인:")
    print(f"src padding mask: {(src == pad_token_id)}")
    print(f"tgt padding mask: {(tgt == pad_token_id)}")

    # EOS 토큰이 마지막에 있는지 확인
    print("\n<EOS> 토큰 확인:")
    print(f"src에서 <eos>의 위치: {(src == eos_token_id).nonzero()}")
    print(f"tgt에서 <eos>의 위치: {(tgt == eos_token_id).nonzero()}")

# 예시로 사용
src = batch['src_input_ids']
tgt = batch['tgt_input_ids']
check_padding_and_eos(src, tgt, tokenizer)


패딩 마스크 확인:
src padding mask: tensor([[False, False, False,  ...,  True,  True,  True],
        [False, False, False,  ...,  True,  True,  True],
        [False, False, False,  ...,  True,  True,  True],
        ...,
        [False, False, False,  ...,  True,  True,  True],
        [False, False, False,  ...,  True,  True,  True],
        [False, False, False,  ...,  True,  True,  True]])
tgt padding mask: tensor([[False, False, False,  ...,  True,  True,  True],
        [False, False, False,  ...,  True,  True,  True],
        [False, False, False,  ...,  True,  True,  True],
        ...,
        [False, False, False,  ...,  True,  True,  True],
        [False, False, False,  ...,  True,  True,  True],
        [False, False, False,  ...,  True,  True,  True]])

<EOS> 토큰 확인:
src에서 <eos>의 위치: tensor([[ 0, 13],
        [ 1, 16],
        [ 2, 12],
        [ 3, 23],
        [ 4, 10],
        [ 5, 16],
        [ 6, 11],
        [ 7, 25]])
tgt에서 <eos>의 위치: tensor([[ 0, 14],
        [ 1, 19],


In [ ]:
#2. 손실 함수에서 NaN 값 문제 확인:

loss_fn = torch.nn.CrossEntropyLoss(ignore_index=1)
optimizer = torch.optim.Adam(transformer.parameters(), lr=0.0001, betas=(0.9,0.98), eps=1e-9)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def check_loss_computation(logits, tgt_out, loss_fn):
    # logits와 tgt_out 크기 확인
    print(f"logits shape: {logits.shape}")
    print(f"tgt_out shape: {tgt_out.shape}")

    # 텐서의 크기가 올바른지 확인
    logits_flat = logits.view(-1, logits.shape[-1])
    tgt_out_flat = tgt_out.view(-1)

    print(f"Flattened logits shape: {logits_flat.shape}")
    print(f"Flattened target shape: {tgt_out_flat.shape}")

    # 손실 계산 시 NaN이 발생하는지 확인
    loss = loss_fn(logits_flat, tgt_out_flat)
    if torch.isnan(loss):
        print("NaN 발생: 손실 계산 중 문제가 있습니다.")
    else:
        print(f"손실 값: {loss.item()}")

src = batch['src_input_ids'].to(device)
tgt = batch['tgt_input_ids'].to(device)

tgt_input=tgt[:-1,:]#target은 마지막 토큰 제외하고 나머지 사용, <EOS> 제외
tgt_out=tgt[1:, :]#target의 첫번째 토큰을 제외한 나머지, <BOS> 제외

#마스크 생성
src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)

# 예시로 사용
logits = model(src, tgt_input, src_mask, tgt_mask, src_padding_mask, tgt_padding_mask, src_padding_mask)
tgt_out = tgt[1:, :]
check_loss_computation(logits, tgt_out, loss_fn)


logits shape: torch.Size([7, 128, 250054])
tgt_out shape: torch.Size([7, 128])
Flattened logits shape: torch.Size([896, 250054])
Flattened target shape: torch.Size([896])
NaN 발생: 손실 계산 중 문제가 있습니다.


/usr/local/lib/python3.10/dist-packages/torch/nn/functional.py:5193: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(


In [ ]:
#loss값 확인
def check_loss_computation(logits, tgt_out, loss_fn):
    # logits와 tgt_out 크기 확인
    print(f"logits shape: {logits.shape}")
    print(f"tgt_out shape: {tgt_out.shape}")

    # logits 값의 범위를 확인하고 clamp 적용
    logits = torch.clamp(logits, min=-10, max=10)
    print(f"Clamped logits 범위: {logits.min().item()} ~ {logits.max().item()}")

    # 텐서의 크기가 올바른지 확인
    logits_flat = logits.view(-1, logits.shape[-1])
    tgt_out_flat = tgt_out.view(-1)

    print(f"Flattened logits shape: {logits_flat.shape}")
    print(f"Flattened target shape: {tgt_out_flat.shape}")

    # 손실 계산 시 NaN이 발생하는지 확인
    loss = loss_fn(logits_flat, tgt_out_flat)
    if torch.isnan(loss):
        print("NaN 발생: 손실 계산 중 문제가 있습니다.")
    else:
        print(f"손실 값: {loss.item()}")

# 예시로 사용
logits = model(src, tgt_input, src_mask, tgt_mask, src_padding_mask, tgt_padding_mask, src_padding_mask)
tgt_out = tgt[1:, :]
check_loss_computation(logits, tgt_out, loss_fn)


logits shape: torch.Size([7, 128, 250054])
tgt_out shape: torch.Size([7, 128])
Clamped logits 범위: nan ~ nan
Flattened logits shape: torch.Size([896, 250054])
Flattened target shape: torch.Size([896])
NaN 발생: 손실 계산 중 문제가 있습니다.


In [ ]:
#3. 출력 로그 안정성 확인
def check_logits_stability(logits):
    # logits 값의 범위를 확인
    max_val = torch.max(logits)
    min_val = torch.min(logits)

    print(f"logits 최대값: {max_val.item()}")
    print(f"logits 최소값: {min_val.item()}")

    # logits 값이 너무 크거나 작은 경우 clamping 적용
    if max_val > 1e4 or min_val < -1e4:
        print("logits 값이 너무 큽니다. torch.clamp()로 조정합니다.")
        logits = torch.clamp(logits, -1e4, 1e4)

    return logits

# 예시로 사용
logits = check_logits_stability(logits)


logits 최대값: nan
logits 최소값: nan


In [ ]:
#4. bleu score 확인
def check_bleu_score(reference, candidate):
    # 참조 문장과 예측 문장이 제대로 되어 있는지 확인
    print("Reference 문장 예시:", reference[0])
    print("Candidate 문장 예시:", candidate[0])

    # BLEU 스코어 계산 시 문제가 없는지 확인
    smoothie = SmoothingFunction().method4
    bleu_score = sentence_bleu([reference], candidate, smoothing_function=smoothie)
    print(f"BLEU-4 스코어: {bleu_score}")

# 예시로 사용
predicted_tokens = logits.argmax(dim=-1)
reference = tgt[1:, :].tolist()
candidate = predicted_tokens.tolist()
check_bleu_score(reference, candidate)


NameError: name 'logits' is not defined

# 학습 진행

In [ ]:
#BLEU score 계산
import nltk
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
smoothie = SmoothingFunction().method4

# BLEU-4 점수 계산 함수
def calculate_bleu(output_ids, target_ids, tokenizer):
    total_bleu = 0
    total_sentences = 0

    for i in range(output_ids.size(0)):  # 배치의 각 문장에 대해 BLEU 계산
        pred_sentence = tokenizer.decode(output_ids[i], skip_special_tokens=True)  # 예측된 문장
        tgt_sentence = tokenizer.decode(target_ids[i], skip_special_tokens=True)  # 실제 문장

        # BLEU-4 계산
        reference = [tgt_sentence.split()]  # 참조 문장
        candidate = pred_sentence.split()  # 예측 문장
        bleu_score = sentence_bleu(reference, candidate, smoothing_function=smoothie)
        total_bleu += bleu_score
        total_sentences += 1

    avg_bleu = total_bleu / total_sentences if total_sentences > 0 else 0
    return avg_bleu

In [ ]:
import torch.nn as nn

#손실 함수에 EOS 토큰 가중치 적용
def create_weighted_loss(vocab_size, eos_token_id, pad_token_id, device):
    weights = torch.ones(vocab_size).to(device)  #GPU로 옮김
    weights[eos_token_id] = 0.1 #<EOS> 토큰에 대한 가중치를 낮춤
    weights[pad_token_id] = 0.0#<PAD> 토큰은 무시

    return nn.CrossEntropyLoss(weight=weights, ignore_index=pad_token_id)


In [ ]:
import torch.cuda.amp as amp
from tqdm import tqdm
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
smoothie = SmoothingFunction().method4

import torch.nn.functional as F

# 손실 함수 및 옵티마이저
#loss_fn = nn.CrossEntropyLoss(ignore_index=tokenizer.pad_token_id)
eos_token_id = tokenizer.eos_token_id
pad_token_id = tokenizer.pad_token_id
loss_fn = create_weighted_loss(vocab_size=tokenizer.vocab_size, eos_token_id=eos_token_id, pad_token_id=pad_token_id, device=DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

# 학습 함수
def train_epoch(model, optimizer, dataloader, loss_fn, device, pbar):
    model.train()  # 모델을 학습 모드로 전환
    total_loss = 0
    total_bleu=0
    scaler = amp.GradScaler()  # Mixed Precision에 필요한 GradScaler 초기화


    for batch in dataloader:
        #데이터 로드
        src = batch['src_input_ids'].to(device).transpose(0,1)
        tgt = batch['tgt_input_ids'].to(device).transpose(0,1)

         #마스크 생성
        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt)

        #옵티마이저 초기화
        optimizer.zero_grad()

        with amp.autocast():
            #모델의 출력 계산
            output = model(src, tgt[:-1, :], None, tgt_mask, src_padding_mask, tgt_padding_mask, src_padding_mask)

            #손실 계산 (output을 [batch_size, sequence_len, vocab_size]로 변환)
            tgt_out = tgt[1:, :]  #<BOS> 토큰 제외
            loss = loss_fn(output.reshape(-1, output.shape[-1]), tgt_out.reshape(-1))

        # 역전파 및 파라미터 업데이트
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        total_loss += loss.item()

        #BLEU-4 스코어 계산
        output_ids = output.argmax(dim=-1).transpose(0,1)#예측 토큰 -> [batch_size, sequence_length]
        avg_bleu = calculate_bleu(output_ids, tgt[1:, :].transpose(0, 1), tokenizer)  # BLEU 계산
        total_bleu += avg_bleu


        #tqdm 업데이트
        pbar.update(1)

        #평균계산
        avg_loss = total_loss/len(dataloader)
        avg_bleu = total_bleu/len(dataloader)

    return avg_loss, avg_bleu



# 평가 함수
def evaluate(model, dataloader, loss_fn, device, pbar):
    model.eval()  # 평가 모드
    total_loss = 0
    total_bleu=0

    with torch.no_grad():  # 평가 시에는 역전파가 필요 없으므로 no_grad 사용
        for batch in dataloader:
            src = batch['src_input_ids'].to(device).transpose(0,1)
            tgt = batch['tgt_input_ids'].to(device).transpose(0,1)

            src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt)

            output = model(src, tgt[:-1, :], None, tgt_mask, src_padding_mask, tgt_padding_mask, src_padding_mask)
            tgt_out = tgt[1:, :]
            loss = loss_fn(output.reshape(-1, output.shape[-1]), tgt_out.reshape(-1))

            total_loss += loss.item()

            #BLEU-4 스코어 계산
            output_ids = output.argmax(dim=-1).transpose(0,1)#예측 토큰 -> [batch_size, sequence_length]
            avg_bleu = calculate_bleu(output_ids, tgt[1:, :].transpose(0, 1), tokenizer)  # BLEU 계산
            total_bleu += avg_bleu


            #tqdm 업데이트
            pbar.update(1)

            #평균계산
            avg_loss = total_loss/len(dataloader)
            avg_bleu = total_bleu/len(dataloader)


    return avg_loss, avg_bleu


#test
def test_model(model, dataloader, loss_fn, device, tokenizer):
    model.eval()
    total_loss = 0  # 손실 누적
    total_bleu = 0  # BLEU-4 스코어 누적
    total_sentences = 0  # 문장 수 누적

    with torch.no_grad():
        for batch in dataloader:
            src = batch['src_input_ids'].to(device).transpose(0, 1)
            tgt = batch['tgt_input_ids'].to(device).transpose(0, 1)

            # 마스크 생성
            src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt)

            with amp.autocast():
                # 모델의 출력 계산
                output = model(src, tgt[:-1, :], None, tgt_mask, src_padding_mask, tgt_padding_mask, src_padding_mask)

                # 손실 계산
                tgt_out = tgt[1:, :]
                loss = loss_fn(output.reshape(-1, output.shape[-1]), tgt_out.reshape(-1))
                total_loss += loss.item()

            # BLEU-4 계산
            output_ids = output.argmax(dim=-1).transpose(0, 1)  # 예측 토큰
            avg_bleu = calculate_bleu(output_ids, tgt[1:, :].transpose(0, 1), tokenizer)  # BLEU 계산
            total_bleu += avg_bleu
            total_sentences += 1

    avg_loss = total_loss / len(dataloader)  # 평균 손실
    avg_bleu_score = total_bleu / total_sentences if total_sentences > 0 else 0  # 평균 BLEU-4 스코어

    return avg_loss, avg_bleu_score


###################

train_dataset = TranslationDataset(ds['train'],tokenizer)
val_dataset = TranslationDataset(ds['validation'],tokenizer)
test_dataset = TranslationDataset(ds['test'],tokenizer)
train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=8, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=8, shuffle=False)

"""
small_train_dataset = TranslationDataset(ds['train'].select(range(200)),tokenizer)
small_val_dataset = TranslationDataset(ds['validation'].select(range(200)),tokenizer)
small_test_dataset =  TranslationDataset(ds['test'].select(range(200)),tokenizer)
train_dataloader = DataLoader(small_train_dataset, batch_size=8)
val_dataloader = DataLoader(small_val_dataset, batch_size=8)
test_dataloader = DataLoader(small_test_dataset, batch_size=8) """


# 에폭 반복문
EPOCHS = 10  # 원하는 에폭 수 설정

for epoch in range(1, EPOCHS + 1):
    print(f"Epoch {epoch}/{EPOCHS}")

    # 훈련 - tqdm 적용
    with tqdm(total=len(train_dataloader), desc=f"Training Epoch {epoch}/{EPOCHS}", unit="batch") as pbar:
        train_loss, train_bleu = train_epoch(model, optimizer, train_dataloader, loss_fn, DEVICE, pbar)
    print(f"  Train loss: {train_loss}, Train Bleu: {train_bleu}")

    # 검증 - tqdm 적용
    with tqdm(total=len(val_dataloader), desc=f"Validation Epoch {epoch}/{EPOCHS}", unit="batch") as pbar:
        val_loss, val_bleu = evaluate(model, val_dataloader, loss_fn, DEVICE, pbar)
    print(f"  Validation loss: {val_loss}, Validation Bleu: {val_bleu}\n")

# 학습이 끝난 후 테스트 데이터로 평가
print("\n\nEvaluating on Test dataset : ")
test_loss, test_bleu = test_model(model, test_dataloader, loss_fn, DEVICE, tokenizer)
print(f"Test loss: {test_loss}, Test BLEU-4: {test_bleu}")

Epoch 1/10


Training Epoch 1/10:   0%|          | 0/3625 [00:00<?, ?batch/s]<ipython-input-9-625c61983a25>:20: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = amp.GradScaler()  # Mixed Precision에 필요한 GradScaler 초기화
<ipython-input-9-625c61983a25>:34: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast():
/usr/local/lib/python3.10/dist-packages/torch/nn/functional.py:5193: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(
Training Epoch 1/10: 100%|██████████| 3625/3625 [14:29<00:00,  4.17batch/s]


  Train loss: 4.5515301590294674, Train Bleu: 0.0700460452948515


Validation Epoch 1/10: 100%|██████████| 127/127 [00:12<00:00, 10.52batch/s]


  Validation loss: 3.427199134676475, Validation Bleu: 0.12189711960692035

Epoch 2/10


...

  Validation loss: 2.1431008980968804, Validation Bleu: 0.19346328193603882

Epoch 10/10


Training Epoch 10/10: 100%|██████████| 3625/3625 [14:28<00:00,  4.17batch/s]


  Train loss: 1.4626528172986262, Train Bleu: 0.15967974172200577


Validation Epoch 10/10: 100%|██████████| 127/127 [00:12<00:00, 10.50batch/s]
<ipython-input-9-625c61983a25>:117: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast():


  Validation loss: 2.1490619914738214, Validation Bleu: 0.20329236082677538



Evaluating on Test dataset : 
Test loss: 2.0736761302947997, Test BLEU-4: 0.200507296805846


In [ ]:
torch.save(model, 'model.pth')

# 평가

In [ ]:
#############################
#번역 작업 수행
def translate(model, sentence, tokenizer, device, max_length=128):
    model.eval()  # 모델을 평가 모드로 전환

    # 입력 문장 토큰화 (소스 언어로 설정된 상태에서)
    inputs = tokenizer(sentence, return_tensors="pt", max_length=max_length, truncation=True, padding="max_length").to(device)

    # 소스 문장을 인코딩
    with torch.no_grad():
        src_mask = torch.zeros((inputs['input_ids'].shape[1], inputs['input_ids'].shape[1]), device=device).type(torch.bool)
        memory = model.encode(inputs['input_ids'].transpose(0, 1), src_mask)

    # 타겟 문장 시작을 <sos> 토큰으로 설정
    tgt_tokens = torch.ones(1, 1).fill_(tokenizer.bos_token_id).type(torch.long).to(device)

    for _ in range(max_length):
        tgt_mask = generate_square_subsequent_mask(tgt_tokens.size(0)).to(device)

        # 디코딩
        with torch.no_grad():
            output = model.decode(tgt_tokens, memory, tgt_mask)
            output = model.generator(output)

        #확인용~##############################################################
        # Softmax 적용하여 확률로 변환##############################
        probabilities = F.softmax(output[-1, :, :], dim=-1)
        # 상위 top_k 후보 확인
        top_k=5
        top_k_probabilities, top_k_indices = torch.topk(probabilities, top_k, dim=-1)

        # 출력 후보들을 확인
        print("\nTop-k predictions:")
        for i in range(top_k):
            token_id = top_k_indices[0, i].item()
            token_prob = top_k_probabilities[0, i].item()
            token = tokenizer.decode([token_id])
            print(f"Token: {token}, Probability: {token_prob:.4f}")
        ##########################################################

        # 가장 가능성 높은 토큰을 선택
        next_token = output.argmax(dim=-1)[-1].item()

        # <eos> 토큰이 나오면 종료
        if next_token == tokenizer.eos_token_id:
            break

        # 예측된 토큰을 타겟 토큰 시퀀스에 추가
        tgt_tokens = torch.cat([tgt_tokens, torch.tensor([[next_token]], device=device)], dim=0)

    # 번역된 토큰을 텍스트로 변환
    translated_sentence = tokenizer.decode(tgt_tokens.flatten(), skip_special_tokens=True)

    return translated_sentence


In [ ]:
sentence='You will face many defeats in life, but never let yourself be defeated.'
translated_sentence = translate(model, sentence, tokenizer, DEVICE)
print(f'\n sentence : {sentence}')
print(f'translated_sentence : {translated_sentence}')


Top-k predictions:
Token: mit, Probability: 0.2635
Token: ht, Probability: 0.0449
Token: ,, Probability: 0.0201
Token: “., Probability: 0.0126
Token: ., Probability: 0.0112

Top-k predictions:
Token: einem, Probability: 0.2661
Token: I, Probability: 0.0587
Token: einer, Probability: 0.0406
Token: Gemüse, Probability: 0.0364
Token: S, Probability: 0.0330

Top-k predictions:
Token: I, Probability: 0.1016
Token: Wagen, Probability: 0.0591
Token: S, Probability: 0.0326
Token: „, Probability: 0.0277
Token: Hand, Probability: 0.0211

Top-k predictions:
Token: ing, Probability: 0.1374
Token: hel, Probability: 0.0241
Token: wagen, Probability: 0.0179
Token: rok, Probability: 0.0178
Token: “., Probability: 0.0122

Top-k predictions:
Token: hel, Probability: 0.1893
Token: “, Probability: 0.0469
Token: “., Probability: 0.0417
Token: hut, Probability: 0.0225
Token: ., Probability: 0.0199

Top-k predictions:
Token: m, Probability: 0.0735
Token: ing, Probability: 0.0480
Token: men, Probability: 0.0

In [ ]:
sentence="I love you."
translated_sentence = translate(model, sentence, tokenizer, DEVICE)
print(f'\n sentence : {sentence}')
print(f'translated_sentence : {translated_sentence}')


Top-k predictions:
Token: mit, Probability: 0.1442
Token: ht, Probability: 0.0353
Token: ,, Probability: 0.0238
Token: voller, Probability: 0.0213
Token: wagen, Probability: 0.0175

Top-k predictions:
Token: einem, Probability: 0.2279
Token: I, Probability: 0.0584
Token: la, Probability: 0.0413
Token: einer, Probability: 0.0363
Token: S, Probability: 0.0319

Top-k predictions:
Token: I, Probability: 0.0851
Token: S, Probability: 0.0449
Token: Hut, Probability: 0.0351
Token: Wagen, Probability: 0.0340
Token: Ru, Probability: 0.0170

Top-k predictions:
Token: ing, Probability: 0.1201
Token: hel, Probability: 0.0266
Token: wagen, Probability: 0.0201
Token: ., Probability: 0.0177
Token: Hut, Probability: 0.0145

Top-k predictions:
Token: hel, Probability: 0.1621
Token: “, Probability: 0.0303
Token: “., Probability: 0.0297
Token: ., Probability: 0.0288
Token: ,, Probability: 0.0236

Top-k predictions:
Token: m, Probability: 0.0895
Token: ing, Probability: 0.0407
Token: er, Probability: 0.0

In [ ]:
#빔서치
import torch.nn.functional as F

def beam_search_decode(model, memory, src_mask, tokenizer, device, beam_width=5, max_length=128):
    model.eval()

    # <sos> 토큰으로 초기화된 빔 리스트
    beams = [(torch.ones(1, 1).fill_(tokenizer.bos_token_id).type(torch.long).to(device), 0)]  # (토큰 시퀀스, 누적 확률)

    for _ in range(max_length):
        new_beams = []

        for tokens, score in beams:
            # 마스킹 생성
            tgt_mask = generate_square_subsequent_mask(tokens.size(0)).to(device)

            # 디코딩
            with torch.no_grad():
                output = model.decode(tokens, memory, tgt_mask)
                output = model.generator(output)

            # 확률 계산 (softmax)
            probabilities = F.log_softmax(output[-1, :], dim=-1)

            # 상위 beam_width 후보 선택
            top_k_probabilities, top_k_indices = torch.topk(probabilities, beam_width, dim=-1)

            # 각 후보에 대해 새로운 빔 생성
            for i in range(beam_width):
                next_token = top_k_indices[i].item()
                new_score = score + top_k_probabilities[i].item()  # 점수 업데이트

                # <EOS> 토큰이면 경로 종료
                if next_token == tokenizer.eos_token_id:
                    return tokens  # 가장 가능성 높은 경로 반환

                # 새로운 토큰 추가된 시퀀스
                new_tokens = torch.cat([tokens, torch.tensor([[next_token]], device=device)], dim=0)
                new_beams.append((new_tokens, new_score))

        # 상위 beam_width개의 빔만 유지
        beams = sorted(new_beams, key=lambda x: x[1], reverse=True)[:beam_width]

    # 가장 높은 점수의 빔 선택
    return beams[0][0]

def translate_sentence_beam_search(model, sentence, tokenizer, device, beam_width=5, max_length=128):
    model.eval()

    # 입력 문장 토큰화
    inputs = tokenizer(sentence, return_tensors="pt", max_length=max_length, truncation=True, padding="max_length").to(device)

    # 소스 문장 인코딩
    with torch.no_grad():
        src_mask = torch.zeros((inputs['input_ids'].shape[1], inputs['input_ids'].shape[1]), device=device).type(torch.bool)
        memory = model.encode(inputs['input_ids'].transpose(0, 1), src_mask)

    # 빔 서치 디코딩
    output_tokens = beam_search_decode(model, memory, src_mask, tokenizer, device, beam_width=beam_width, max_length=max_length)

    # 번역된 토큰을 텍스트로 변환
    translated_sentence = tokenizer.decode(output_tokens.flatten(), skip_special_tokens=True)

    return translated_sentence


In [ ]:
sentence="I love you."
translated_sentence = translate_sentence(model, sentence, tokenizer, DEVICE)
print(f'\n sentence : {sentence}')
print(f'translated_sentence : {translated_sentence}')


 sentence : I love you.
translated_sentence : mit einem Iinghelm.
